<strong>NOTE:</strong> When the normalizer is adapted on X_train, the loss function decreases much faster than when the normalizer is adapted on all the features.

In [1]:
import pandas as pd
import tensorflow as tf

dataframe = pd.read_csv("Energy.csv")

In [2]:
### Seperate Data
features = dataframe.iloc[:, :-1]
label = dataframe.iloc[:, -1]

In [3]:
### Split Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features.values, label.values, test_size = 0.2)

In [4]:
### Normalize Data
from sklearn.preprocessing import StandardScaler
training_scaler = StandardScaler()
testing_scaler = StandardScaler()

X_train = training_scaler.fit_transform(X_train)
X_test = testing_scaler.fit_transform(X_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7654, 4), (1914, 4), (7654,), (1914,))

In [5]:
### Adjust the shape of y to make it compatible with the shape of X
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [6]:
### Create Normalizer to add a Normalizing layer
from keras.layers import Normalization
training_features = tf.constant(features)
# training_features = tf.constant(X_train)
normalizer = Normalization()
normalizer.adapt(training_features)

In [7]:
### Build Model
from keras import Sequential
from keras.layers import InputLayer, Dense
from keras.losses import MeanAbsoluteError
from keras.metrics import MeanSquaredError, R2Score
from keras.optimizers import Adam

ANN = Sequential()
ANN.add( InputLayer( shape = (4,) ) )
ANN.add( normalizer )
ANN.add( Dense( units = 25, activation = "relu" ) )
ANN.add( Dense( units = 25, activation = "relu" ) )
ANN.add( Dense( units = 1 ) )
ANN.compile( optimizer = Adam(), loss = MeanAbsoluteError(), metrics = [MeanSquaredError(), R2Score()])

### Train
ANN.fit( X_train, y_train, epochs = 100)

Epoch 1/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 389.3153 - mean_squared_error: 165429.7500 - r2_score: -568.3644
Epoch 2/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.7766 - mean_squared_error: 304.4073 - r2_score: -0.0442
Epoch 3/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.6714 - mean_squared_error: 294.5750 - r2_score: -0.0245
Epoch 4/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 14.9014 - mean_squared_error: 304.7369 - r2_score: -0.0316
Epoch 5/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.6127 - mean_squared_error: 297.5396 - r2_score: -0.0352
Epoch 6/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.5975 - mean_squared_error: 299.1243 - r2_score: -0.0369
Epoch 7/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.6706 - mean_squared_error: 299.0482 - r2_score: -0.0243
Epoch 8/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.4962 - mean_squared_error: 289.7235 - r2_score: -0.0041 
Epoch 9/100
240/240 ━━━━━━━━━━━━━

In [8]:
### Evaluate
ANN.evaluate(X_test, y_test)

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.2499 - mean_squared_error: 42.1590 - r2_score: 0.8547


[5.237518310546875, 42.42148208618164, 0.8547354936599731]